# Proof of Stake

Proof of Stake renueva todo el sistema de "carrera de mineros" a "grupo de validadores sin esperanzas." Proof of Stake fue creado para solventar la principal problematica de Proof of Work (Gasto energetico).

Ya no existen los mineros, ahora los nodos seran "validadores y forjadores". En este consenso, "minar un bloque" pasa a ser "forjar un bloque", significan lo mismo. 
- El forjador se encarga de verificar las transacciones y agruparlas en un bloque, firmar ese bloque con su llave privada y enviarlo a la red.
- Los Validadores se encargan de revisar que el trabajo del forjador este correcto.
- Cuando le bloque haya sido validador. El forjador anexa el bloque a la Blockchain.

Antes, los mineros provaban su validez en la red con su trabajo (de ahi viene su nombre). Ahora, prueban su validez con su "liquidez". Cada nodo pone su dinero en la red para provar que es de confianza; si el nodo llegara a querer danar la identidad de la red, su dinero (que no es una cantidad pequena) seria destruido.

Se dice que son nodos Trustless, debido a que como nadie quiere ser un nodo corrupto y perder todo su dinero, nadie confia en nadie y se revisa el trabajo de todos.

En este consenso, entre mas dinero tenga un nodo puesto como "seguro" en la red, mas probabilidad tiene de ser el siguiente forjador del bloque. 

Cada que se escoje un nuevo forjador, se escojen tambien un numero determinado de validadores. 

Nos vamos a centrar en todo este mecanismo, poniendo un ejemplo, para despues ver como se implementa en una funcion de nuestra cadena de bloques.

Vamos a ir desarrollando poco a poco, clase por clase

 ## Nodos

Un nodo que esta participando para ser elegido como validador o como forjador, tiene que tener si o si una cuenta en la que recibiria las recompensas de la red. Una cuenta puede usar la red sin la necesidad de ser un validador. Un validor es un validador por dos motivos:
1. Por que el usuario asi lo desea
2. Por que cuenta con dinero suficiente para demostrar que es de confianza.

En Proof of Stake, se necesita un "minimo" de dinero virtual para ser validor de la red.

Podemos concluir que: Si tienes una cuenta en la Blockchain, y tienes minimo N cantidad de dinero virtual, puedes ser un validador.

Para la blockchain que vamos a desarrollar, vamos a crear una nueva clase llamada Validator. Esta clase va a tener dos atributos:
- "Tokens" por cuenta.
- La direccion de la cuenta que es Validador.

In [1]:
from tokens import Token
from account import Account

class Validator():
    def __init__(self, account):
        self.account = account
        self.tokens = []

    def set_tokens(self, total_coins):
        """Funcion que por cada moneda en el balance, se instancia un nuevo token en la cuenta."""
        for every_coin in range(0, total_coins):
            self.tokens.append(Token(self))

    def get_tokens(self):
        """Funcion que devuelve el numero de tokens en la cuenta."""
        return len(self.tokens)

Los **tokens** son objetos dentro de una blockchain que puede tener una utilidad dentro de la misma. En este caso, nos van a ayudar para resolver el algoritmo de seleccion de forjador dentro de un grupo de validadores. Vamos a crear una clase Token.

## Tokens

In [2]:
class Token():
    def __init__(self, owner):
        self.owner = owner

La clase Token va a tener una funcion muy sencilla, pero muy importante. Tener un dueno. 

## Como se ve la clase Forjador y Testigo?

### Clase Forger

Un forjador es encargado de crear un bloque y anexarlo a red, asi como:
1. Primero agrupa las transacciones y las verifica.
2. Crea un bloque y anexa las transacciones.
3. Firma el bloque con su llave privada y lo envia a los validadores del bloque.
4. Los demas validadores ahora seran testigos, y su trabajo sera atestiguar que el nodo hizo bien su trabajo.
4. Si todos llegan a un consenso, el forjador tendra permiso para anexar el nodo a la red.

La clase que vamos a usar en nuestra Blockchain va a necesitar los siguientes atributos:
- validator: Guarda la direccion de la cuenta que esta realizando la transaccion.
- block: Variable que almacena el bloque que se crea.
- block_signature: Variable que almacena la firma de bloque hecha por el forjador.

In [27]:
class Forger():
    def __init__(self, validator):
        self.validator = validator # Cuando se crea un Forger, se vincula con la cuenta del validador. 
        self.block = None # Bloque creado por el forjador
        self.block_signature = None # Firma del bloque 

Lo primero que hace un forjador es agrupar las transacciones en espera y verificarlas.  

In [28]:
def verify_tx(self, holding_tx):
        """Funcion en la que el forjador verifica que las transacciones en espera sean
        validas para incluirlas en el bloque. La verificacion es igual que en PoW."""
        print('El forjador esta verificando las tx.')
        verified_tx = []
        for tx in holding_tx:
            if tx.verify_signature(): # Utiliza el metodo de la clase Transaction para su propia firma.
                print(tx.to_dict())
                verified_tx.append(tx) # Si la firma se verifica, se anexa a una nueva lista llamada verified_tx
        return verified_tx

Una vez listas las transacciones, el forjador crea un objeto bloque.

In [32]:
from block import Block

def create_a_block(self, _previous_hash, _verified_tx, _block_number):
    self.block = Block(_previous_hash, _verified_tx, _block_number)

Con el bloque creado, y con las transacciones verificadas y almacenadas, el forjador firma el bloque. El proceso es similar a cuando se firma una transaccion.

In [35]:
    def sign_block(self) -> bool:
        """El forjador recibe un objeto bloque, lo firma con su llave privada."""
        print("### FIRMANDO EL BLOQUE")
        # Se anade el validador al bloque.
        self.block.forger = self # Antes de firmarse el bloque, se guarda en el bloque la informacion de quien lo forjo.
        block_header = json.dumps(self.block.get_block_header_pos()).encode() # Se obtiene el header del bloque para pasarlo a formato JSON
        block_hashed = SHA256.new(block_header) # El bloque pasa por un algoritmo SHA256
        #
        signer = self.validator.account.signer # Cada forjador es una cuenta con el titulo de forjador, por lo tanto, tiene su propia llave privada 
        signature = signer.sign(block_hashed)
        self.block.hash = block_hashed.hexdigest() # Se almacena dentro del bloque su propio hash
        self.block_signature = signature # El forjado guarda consigo su firma del bloque
        print('### BLOQUE FIRMADO Y CON SU HASH ---')

    def broadcast_block(self):
        """Funcion que retorna el bloque creado del forjador."""
        return self.block

Con la transaccion firmada, solo queda mandarla a la red.

In [36]:
def broadcast_block(self):
    return self.block

### Clase Attestor

Los Attestors (Testigos en espanol) son los encargados de verificar el trabajo del forjador; "atestiguan" que todo este bien. Su metodologia iria algo asi:
- Reciben el bloque del forjador.
- Revisan que la firma del bloque sea correcta.
- y si no hay nada raro, y mas del 75% de los validadores estan de acuerdo, el bloque puede pasar a ser parte de la red.

In [39]:
class Attestor():
    def __init__(self, validador) -> None:
        self.validator = validador # Como atributos, solo necesitan tener un lugar en memoria para guardar la cuenta.

Como metodos solo vamos a necesitar una funcion que revise el bloque.

In [46]:
    def check_block(self, block):
        """Funcion que recibe un bloque, lo verifica y retorna un valor booleano si la revision es exitosa."""
        # Asi como se hacian las verificaciones de transacciones, se vuelve a obtener el hash del contenido (bloque en este caso)
        block_header = json.dumps(block.get_block_header_pos()).encode() 
        block_hashed = SHA256.new(block_header)
        # recordemos que se usa el verificador quien forjo el bloque.
        verifier = block.forger.validator.account.verifier
        try:
            verifier.verify(block_hashed, block.forger.block_signature)
            print(f"{self.validator.account.nickname}: Confirmo que la firma del bloque es correcta.")
            return True
        except:
            print("La firma es invalida.")
            return False
        #

En este archivo tambien vamos a incluir una variable Attestors (en plural.). Esta clase nos va a ayudar a contener a todos los testigos disponibles del bloque, y llegar a un acuerdo entre ellos.

Como atributos solo vamos a tener una variable group, la cual es va a encargar de agrupar a todos los validadores en una lista e instanciarlos como objeto Attestor.

In [49]:
class Attestors():
    def __init__(self, validators):
        self.group = [Attestor(validator) for validator in validators] 

Esta clase va a contar con un metodo llamado Attest, el cual revisa los votos de los testigos de la transaccion, obtiene sus confirmaciones, y llega al consenso.

In [54]:
    def attest(self, block):
        """Funcion que recibe un bloque, y va pasando a cada attestor a revisar el bloque. Si las confirmaciones son suficientes, regresa un valor True."""
        confirmations = [] # Variable que almacena las confirmaciones de los testigos
        for attestor in self.group: # Por cada testigos en el grupo de todos los testigos
            confirmations.append(attestor.check_block(block)) # Anexa su respuesta a las confirmaciones
        print('-----------------')
        # Se hace una division para cononcer cuantos votos minimos necesitamos, si el objetivo es superar el 75%
        minimun_votes = len(self.group) // 1.25
        if sum(confirmations) > minimun_votes: # Si las confirmaciones son mayores a un porcentaje designado...
            print('Confirmaciones suficientes para anadir el bloque.')
            return True
        else:
            print('Confirmaciones insuficientes para anadir el bloque.')
            return False

## Algoritmo de seleccion de Forjador y Testigos.

Cada Blockchain que utilice el consenso Proof of Stake tiene un algoritmo distinto; algunos dan mas beneficio a los que casi no participan, algunas penalizan por ciertas acciones u otros dan prioridad por la antiguedad del nodo, pero en todos, la que mayor probabilidad da es tener mas dinero dentro de la red como asegurador de tu trabajo, asi que en esta blockchain unicamente vamos a tomar en cuenta ese.

In [8]:
from account import Account, Validator

validadores_del_bloque = {} # validadores del bloque confirmador
total_stacked = 0 # dinero que se almacena de los validadores

# Vamos a crear 5 cuentas. Cada una de ellas se va a instanciar como un validador.
# Si bien sabemos que el dinero no sale de la nada, ponerles un balance nos ayuda para ejemplificar 
# algo mas adelante...

charles = Account(350, 'charles')
edwin = Account(500, 'edwin')
oliver = Account(200, 'oliver')
erick = Account(90, 'erick')
sonia = Account(275, 'sonia')

# Las juntamos en una variable
lista_cuentas = [charles, edwin, oliver, erick, sonia]

# Digamos que todos quieren ser validadores, solo van a pasar aquellos que tengan mas de 100 de balance en su cuenta.
# Lo primero es hacer ese filto
for cuenta in lista_cuentas: # Bucle for que recorre cada una de las cuentas
    if cuenta.balance >= 100: # Si tiene 100 o mas de balance puede ser validador
        # Instanciar un nuevo objeto Validator.
        new_validator = Validator(cuenta) 
        
        # Variable que almacena el dinero que se va a intercambiar por tokens
        account_money = int(new_validator.account.balance)
        
        # A traves del objeto Validator, podemos acceder al objeto Account, y despues al atributo balance para restarle lo que gasto en tokens.
        new_validator.account.balance -= account_money
        
        # Se utiliza al funcion set_tokens para instanciar un numero determinado de tokens en la cuenta.
        # Si el usuario tiene 200 de balance, se van a intercambiar por 200 tokens.
        new_validator.set_tokens(account_money) 

        # Se almacena en un diccionario el validador y la cuenta.
        validadores_del_bloque.update({new_validator: account_money})
        
        # Se suma al total stackeado de la red el balance que se cambio por tokens
        total_stacked += account_money

# Al final vamos a tener una lista de validadores. (Menos Erick)
validadores_del_bloque, total_stacked

({<account.Validator at 0x1850be5d400>: 350,
  <account.Validator at 0x1850becb790>: 500,
  <account.Validator at 0x1850beef670>: 200,
  <account.Validator at 0x1850bef7310>: 275},
 1325)

In [ ]:
# Su balance tambien fue restado.
for keys in validadores_del_bloque.keys():
    print(f"{keys.account.nickname}, {keys.account.balance}")

charles, 0
edwin, 0
oliver, 0
sonia, 0


Una vez que tenemos la lista neta de los validadores que van a formar parte de la forja y validacion del bloque, toca hacer un sorteo.

El mecanismo con el cual se escoje es por probabilidades y es muy similar a una tombola.

#### Como desarrollar el algoritmo?
Dentro de esta celda, se va a disenar un algoritmo que escoja al siguiente forger, el siguiente ejemplo fue una implementacion propia, pero cada blockchain puede variar.  Veamoslo:
- Cada validador tiene una cantidad finita de tokens. 
- Se van a ingresar los tokens de todos los validadores que van a participar en el sorteo, en una sola lista.
- Cada token tiene un dueño, por lo tanto, se sabe de quien es.
- Cada ticket que ingresaron da una la posibilidad de ser el siguiente forjador del bloque.
- La lista se revuelve y se escoje un elemento al azar.
- El dueno del ticket ganador pasa a ser el forjador del nuevo bloque, y los no ganadores a ser los validadores.

In [71]:
from random import sample, choice
from forge import Forge


# 1.- Lista que almacenara todos los tickets de la "rifa".
pool = []
# 2.- Bucle que recorrera a cada validador, y añadara sus tokens a la lista general.
for validator in validadores_del_bloque.keys():
    pool += validator.tokens
# 3.- Se revuelve la lista. (Como si fuera un sorteo.)
print('Acumulando los tokens de los validadores en el servidor actual...')
print(len(pool), '- tokens acumulados.')
print('Revolviendo la lista...')
pool = sample(pool, len(pool))
print('Lista revuelta!')

# Bucle que valida que los tokens se hayan incluido bien
contador = 0
for validator in validadores_del_bloque.keys():  # Por cada validador, en la lista de validadores
    for token in pool: # Por cada ticken en el pool 
        if token.owner.account.nickname == validator.account.nickname: # Si el token pertenece al usuario, se suma uno en el contador
            contador += 1
    print(validator.account.nickname, contador) # Se imprimen los tickets de cada validador 
    contador = 0
# Fin de la validacion

ticket_winner = choice(pool)
forger = Forge(ticket_winner.owner)
print(f'El forjador del nuevo bloque sera... {forger.validator.account.nickname}')

# los validadores no ganadores del sorteo pasan a ser testigos.
# se remueve el forjador, asi solo quedan los testigos
validadores_sin_el_forjador = validadores_del_bloque.copy() # Se hace una copia de la lista que contiene a los validadores de la red
validadores_sin_el_forjador.pop(forger.validator) # Se elimina al forjador de esta transaccion
"no ganadores",[validator.account.nickname for validator in validadores_sin_el_forjador]

Acumulando los tokens de los validadores en el servidor actual...
1325 - tokens acumulados.
Revolviendo la lista...
Lista revuelta!
charles 350
edwin 500
oliver 200
sonia 275
El forjador del nuevo bloque sera... sonia


('no ganadores', ['charles', 'edwin', 'oliver'])

Aquellos que no ganaron el sorteo, pasan a ser "testigos".

In [72]:
from attestor import Attestor, Attestors

# Para instanciar al grupo de testigos, necesitamos usar la clase Attestors
attestors = Attestors(validadores_sin_el_forjador)

Ya tenemos a nuestro forjador y nuestro testigo. Que seguiria?

#### Procedimiento del Forjador y del Testigo

Lo primero que sucede es que el Forjador recibe un bloque y verifica sus transacciones.



In [142]:
from account import Account
from transaction import Transaction
from block import Block

# Instanciamos un bloque previo al actual para efectos de practica
cadena_de_bloques = [Block('0', [], 0)]

# Vamos a crear una transaccion entre dos cuentas.
tx = Transaction(Account(100, 'pedro'), 10, Account(100, 'maria'))

# Cada transaccion se firma con la llave privada del que envia la transaccion.
tx.sign_transaction()

# Se anade la tx a una lista de espera
holding_tx = [tx]

# El forjador verifica las transacciones en la lista
verified_tx = forger.verify_tx(holding_tx)

# El forjador crea un bloque 
forger.create_a_block(block.hash, verified_tx, block.block_number + 1)

# El forjador firma el bloque
forger.sign_block()

# El forjador envia el bloque a la red
bloque_firmado = forger.broadcast_block()

if attestors.attest(bloque_firmado):
    print('TODO CORRECTO, LLEGAMOS A UN ACUERDO')
    cadena_de_bloques.append(bloque_firmado)
cadena_de_bloques

Firmando transaccion...
El forjador esta verificando las tx.
Verificando la firma de la transaccion...
La firma es valida.
{'sender': 'pedro', 'recipient': 'maria', 'value': 10, 'time': '20/09/2022 16:20:48'}
### FIRMANDO EL BLOQUE
### BLOQUE FIRMADO Y CON SU HASH ---
charles: Confirmo que la firma del bloque es correcta.
edwin: Confirmo que la firma del bloque es correcta.
oliver: Confirmo que la firma del bloque es correcta.
-----------------
Confirmaciones suficientes para anadir el bloque.
TODO CORRECTO, LLEGAMOS A UN ACUERDO


[<block.Block at 0x1850d52d100>, <block.Block at 0x1850d429370>]